<a href="https://colab.research.google.com/github/valerio-unifei/ECOM09A/blob/main/ECOM09A_2024_1_Atividade_2_Etapa_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECOM09A Atividade 2 Etapa 1

(10/05/2024) Realizar o procedimento do notebook:

https://www.kaggle.com/code/marlesson/vocabulary-analysis-word2vec


## Dataset

In [1]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download 'marlesson/news-of-the-site-folhauol'

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/marlesson/news-of-the-site-folhauol
License(s): CC0-1.0
 90% 169M/187M [00:01<00:00, 121MB/s]
100% 187M/187M [00:01<00:00, 115MB/s]


In [3]:
!unzip -qq news-of-the-site-folhauol.zip
!rm news-of-the-site-folhauol.zip

### Carregando os artigos do jornal

In [11]:
import pandas as pd
df=pd.read_csv('articles.csv',encoding="utf8")
print('(linhas, colunas):',df.shape)
df.head()

(linhas, colunas): (167053, 6)


,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...


### Ajustando o Dataset para aprendizado

In [12]:
print('Linhas:',len(df))
df.drop_duplicates(subset=["text"],inplace=True)
print('Removendo duplicadas em "text":',len(df))
df.drop(['subcategory','link','date'],axis=1,inplace=True)
print('Removendo subcategory')
df.dropna(inplace=True)
print('Removendo nulos:',len(df))
df.reset_index(drop=True,inplace=True)
print('Resetando index')
df.head()

Linhas: 167053
Removendo duplicadas em "text": 165902
Removendo subcategory
Removendo nulos: 165901
Resetando index


,title,text,category
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,poder
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",ilustrada
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,poder
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,ilustrada
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",mercado


### Convertendo categorias em campo categorizado

In [20]:
df['category'].value_counts()

category
poder                           22005
colunas                         21499
mercado                         20936
esporte                         19723
mundo                           17123
cotidiano                       16952
ilustrada                       15604
opiniao                          4523
paineldoleitor                   4008
saopaulo                         3953
tec                              2205
educacao                         2113
tv                               2065
turismo                          1902
ilustrissima                     1377
ciencia                          1331
equilibrioesaude                 1311
sobretudo                        1052
bbc                               979
folhinha                          875
empreendedorsocial                841
comida                            828
asmais                            547
ambiente                          490
seminariosfolha                   378
serafina                          331
o-m

Olha o problema acima, temos classes de notícias com apenas 1, 3, 8 até 400 notícias apenas no montante de 167.000, isso demonstra a dificuldade de trabalhar com um dataset mesmo ajustado com problemas de balanceamento de classes.

In [16]:
df['category'] = df['category'].replace('guia-de-livros-discos-filmes','guia-de-livros-filmes-discos')
alvo, valores = pd.factorize(df['category'],sort=True)
df['target'] = alvo
list(enumerate(valores))

[(0, 'ambiente'),
 (1, 'asmais'),
 (2, 'banco-de-dados'),
 (3, 'bbc'),
 (4, 'bichos'),
 (5, 'cenarios-2017'),
 (6, 'ciencia'),
 (7, 'colunas'),
 (8, 'comida'),
 (9, 'cotidiano'),
 (10, 'dw'),
 (11, 'educacao'),
 (12, 'empreendedorsocial'),
 (13, 'equilibrioesaude'),
 (14, 'especial'),
 (15, 'esporte'),
 (16, 'euronews'),
 (17, 'folhinha'),
 (18, 'guia-de-livros-filmes-discos'),
 (19, 'ilustrada'),
 (20, 'ilustrissima'),
 (21, 'infograficos'),
 (22, 'mercado'),
 (23, 'mulher'),
 (24, 'multimidia'),
 (25, 'mundo'),
 (26, 'o-melhor-de-sao-paulo'),
 (27, 'ombudsman'),
 (28, 'opiniao'),
 (29, 'paineldoleitor'),
 (30, 'poder'),
 (31, 'rfi'),
 (32, 'saopaulo'),
 (33, 'seminariosfolha'),
 (34, 'serafina'),
 (35, 'sobretudo'),
 (36, 'tec'),
 (37, 'topofmind'),
 (38, 'treinamento'),
 (39, 'treinamentocienciaesaude'),
 (40, 'turismo'),
 (41, 'tv'),
 (42, 'vice')]

## Realizando treinamento com Sklearn ML Clássico (fora do enunciado)

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

cVec = CountVectorizer()
X = cVec.fit_transform(df['text'].values)

tfid = TfidfTransformer()
X = tfid.fit_transform(X)
print('X:',X.shape)

y = df['target'].values

X: (165901, 342236)


In [23]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier, RidgeClassifier

classificador = [
    MultinomialNB(),
    SGDClassifier(random_state=42),
    RidgeClassifier(random_state=42),
]

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score
from tqdm.notebook import tqdm

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

res = []
for c in tqdm(classificador):
  c.fit(X_train, y_train)
  y_pred = c.predict(X_test)

  dn={
      'metodo':c.__class__.__name__,
      'acurárica':accuracy_score(y_test, y_pred),
      'bal_acc':balanced_accuracy_score(y_test, y_pred),
      'f1':f1_score(y_test, y_pred,average='macro'),
      }
  res.append(dn)

rdf = pd.DataFrame(res)

  0%|          | 0/3 [00:00<?, ?it/s]

In [24]:
rdf

,metodo,acurárica,bal_acc,f1
0,MultinomialNB,0.643893,0.137882,0.134050
1,SGDClassifier,0.812091,0.335836,0.367537
2,RidgeClassifier,0.850607,0.390965,0.419639
